In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/preprocessedtrain/preprocessesTrain.csv
/kaggle/input/its-a-fraud/train.csv
/kaggle/input/its-a-fraud/test.csv
/kaggle/input/its-a-fraud/mock_submission.csv
/kaggle/input/testData/preprocessedTest.csv


In [3]:
def changeDType(df,flag=False):
    
    if(flag):
        numericDtype = ['int32','int64','float64','float32']
    
    for i in df.columns:
        if (df[i].dtype == 'int64' or df[i].dtype == 'int32'):
            df[i] = pd.to_numeric(df[i],downcast='integer')
        
        if (df[i].dtype == 'float64' or df[i].dtype == 'float32'):
            df[i] = pd.to_numeric(df[i],downcast='float')

In [4]:
dfTest = pd.read_csv('../input/testData/preprocessedTest.csv');
dfTrain = pd.read_csv('../input/preprocessedtrain/preprocessesTrain.csv');

In [6]:
changeDType(dfTest)
changeDType(dfTrain)

In [7]:
from sklearn.model_selection import train_test_split,cross_val_score,StratifiedKFold
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn import metrics

In [22]:
def naiveBayesGaussian(df,isTestData=False,model=None):
    
    if(isTestData == False):
        train,test = train_test_split(df,stratify=df['isFraud'],test_size = 0.3,random_state = 9)
    else:
        train,test = train_test_split(df,test_size = 0.3,random_state = 9)
        
    nb = GaussianNB()
    
    if(isTestData):
        x = df
        x = x - x.min()/x.max()-x.min()
        y_pred = model.predict(x)
        result = pd.DataFrame(y_pred)
        result.columns = ['isFraud']
        result.to_csv('./result_nb.csv')
        print("Result file saved")
    else:
        x = train.drop(['isFraud'],axis = 1)
        y = train['isFraud']
    
        x = x - x.min()/x.max()-x.min()

        nb.fit(x,y)
    
        xt = test.drop(['isFraud'],axis=1)
        yt = test['isFraud']
    
        print("Accuracy Score : " ,nb.score(xt,yt))
        prob = nb.predict_proba(xt)[:,1]
    
        auc = metrics.roc_auc_score(yt,prob)
        print("AUC : ",auc)
        return nb

In [23]:
def logisticRegTrain(df):
    
    train, test = train_test_split(train, test_size=0.25, random_state=9)
    model = LogisticRegression(solver='liblinear',random_state=9)
    
    x = train.drop(['isFraud'],axis = 1)
    y = train['isFraud']
    
    x = x - x.min()/x.max()-x.min()
    
    model.fit(x,y)
    print("Score : ",model.score(x, y))
    
    y_pred_proba = model.predict_proba(x)[::,1]

    auc = metrics.roc_auc_score(y, y_pred_proba)
    print("AUC : ",auc)

    return model

In [24]:
model = naiveBayesGaussian(dfTrain)

Accuracy Score :  0.8187661331956634
AUC :  0.7802366608532652


In [25]:
naiveBayesGaussian(dfTest,True,model)

Result file saved
